In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print ("Training set: ", train_dataset.shape, train_labels.shape)
    print ("Validation set: ", valid_dataset.shape, valid_labels.shape)
    print ("Test set: ", test_dataset.shape, test_labels.shape)

Training set:  (200000, 28, 28) (200000,)
Validation set:  (10000, 28, 28) (10000,)
Test set:  (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    datasets = dataset.reshape([-1, image_size*image_size])\
                 .astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None])\
                 .astype(np.float32)
    return datasets, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print ("Training set: ", train_dataset.shape, train_labels.shape)
print ("Validation set: ", valid_dataset.shape, valid_labels.shape)
print ("Test set: ", test_dataset.shape, test_labels.shape)
    
    

Training set:  (200000, 784) (200000, 10)
Validation set:  (10000, 784) (10000, 10)
Test set:  (10000, 784) (10000, 10)


In [4]:
train_size = train_dataset.shape[0]
train_feature_size = train_dataset.shape[1]
label_class_size = train_labels.shape[1]
size_h1 = 1024

graph = tf.Graph()
with graph.as_default():
    
    tf_train_x = tf.placeholder(tf.float32, shape=[None, train_feature_size])
    tf_train_y = tf.placeholder(tf.float32, shape=[None, label_class_size])
    
    w1 = tf.Variable(
        tf.truncated_normal([image_size*image_size, size_h1]))
    b1 = tf.Variable(tf.zeros([size_h1]))
    
    h1 = tf.nn.relu(tf.matmul(tf_train_x, w1)+b1)
    
    w2 = tf.Variable(
        tf.truncated_normal([size_h1, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(h1, w2)+b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_y, logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5)
    optimizer = optimizer.minimize(loss)
    
    y_pred = tf.nn.softmax(logits)

In [5]:
def accuracy(predictions, labels):
    return (100.* np.sum( np.argmax(predictions, 1) == np.argmax(labels, 1))
        / predictions.shape[0])

In [10]:
train_size = train_dataset.shape[0]
num_steps = 5001
batch_size = 100
num_batch = int(train_size/batch_size)
offset = 0

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    print ("Initialized")
    for step in range(num_steps):
        batch_x = train_dataset[offset:offset+batch_size,:]
        batch_y = train_labels[offset:offset+batch_size,:]
        offset += batch_size
        if ( offset >= train_size ): offset = 0
        feed_dict = {tf_train_x: batch_x, tf_train_y: batch_y}
        _, l, predictions = sess.run([optimizer, loss, y_pred], feed_dict=feed_dict)
        if ( step % 500 == 0 ):
            print ("Minibatch loss at step %d: %f" % (step, l))
            print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_y))
            valid_acc = accuracy( sess.run(y_pred, feed_dict={\
                tf_train_x: valid_dataset, tf_train_y: valid_labels}), valid_labels)                                
            print ("Validation accuracy: %.1f%%" % valid_acc)
    test_acc = accuracy( sess.run(y_pred, feed_dict={\
        tf_train_x: test_dataset, tf_train_y: test_labels}), test_labels)
    print ("Test accuracy: %.1f%%" % test_acc)

Initialized
Minibatch loss at step 0: 298.068634
Minibatch accuracy: 13.0%
Validation accuracy: 23.1%
Minibatch loss at step 500: 6.871636
Minibatch accuracy: 81.0%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 8.021955
Minibatch accuracy: 83.0%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 10.610357
Minibatch accuracy: 81.0%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 5.978865
Minibatch accuracy: 76.0%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 1.671727
Minibatch accuracy: 83.0%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 3.073645
Minibatch accuracy: 80.0%
Validation accuracy: 81.1%
Minibatch loss at step 3500: 3.125552
Minibatch accuracy: 84.0%
Validation accuracy: 82.2%
Minibatch loss at step 4000: 5.684232
Minibatch accuracy: 78.0%
Validation accuracy: 82.4%
Minibatch loss at step 4500: 1.325851
Minibatch accuracy: 82.0%
Validation accuracy: 83.1%
Minibatch loss at step 5000: 0.929994
Minibatch accuracy: 86.0%
Validation accu